<a href="https://colab.research.google.com/github/Ahmethan96/Data_science/blob/main/Energy_analysis_of_wind_turbine_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import glob
from datetime import datetime
%matplotlib inline

In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
!pwd

/content


In [101]:
data_frame = pd.read_excel('/AL_WIND_07_12.xlsx', index_col= 'DateTime')
data_frame.head(10)

,Air temperature | ('C),Pressure | (atm),Wind speed | (m/s),Wind direction | (deg),Power generated by system | (kW)
DateTime,,,,,
2007-01-01 00:00:00,10.926,0.979103,9.014,229,33688.10
2007-01-01 01:00:00,9.919,0.979566,9.428,232,37261.90
2007-01-01 02:00:00,8.567,0.979937,8.700,236,30502.90
2007-01-01 03:00:00,7.877,0.980053,8.481,247,28419.20
2007-01-01 04:00:00,7.259,0.979867,8.383,256,27370.30
2007-01-01 05:00:00,6.570,0.979884,8.256,261,25805.90
2007-01-01 06:00:00,5.897,0.980318,6.476,265,11546.80
2007-01-01 07:00:00,5.109,0.980597,5.906,271,8360.76
2007-01-01 08:00:00,4.413,0.980711,5.557,269,6426.64


Setting an index in a DataFrame serves several important purposes in data analysis:

Data Labeling: The index provides a way to uniquely label and identify rows in the DataFrame. It acts as a row identifier and allows for efficient and direct access to specific rows based on their index values.

Enhanced Data Retrieval: Setting an index improves data retrieval performance, especially for large datasets. When an index is defined, Pandas can quickly locate and access rows based on index values, making data retrieval more efficient.

Data Alignment: When performing operations between two DataFrames or when joining DataFrames, having a well-defined index ensures that data aligns correctly. Pandas aligns data based on the index, making it easier to combine and manipulate data from different sources.

Time Series Data: For time series data, setting a datetime index allows for time-based operations, resampling, and slicing based on specific time periods.

Grouping and Aggregating Data: An index can be used for grouping data based on specific criteria, and it allows for efficient data aggregation using functions like groupby.

Data Integrity: Having a unique index helps maintain data integrity, ensuring that rows are uniquely identifiable and preventing duplicate or ambiguous entries.

Merging and Joining Data: When merging or joining multiple DataFrames, setting a common index allows for easy and accurate data alignment.

Hierarchical Indexing: Setting a multi-level index (hierarchical index) enables organizing and working with higher-dimensional data, providing a more structured representation of the data.

In summary, setting an index in a DataFrame is a fundamental step in data analysis that allows for efficient data retrieval, data alignment, grouping, and various data manipulation operations. It improves the overall usability and integrity of the data, making it easier to work with and analyze.

In [102]:
data_frame.dtypes

Air temperature | ('C)              float64
Pressure | (atm)                    float64
Wind speed | (m/s)                  float64
Wind direction | (deg)                int64
Power generated by system | (kW)    float64
dtype: object

In [103]:
data_frame.index.dtype

dtype('<M8[ns]')

**dtype('<M8[ns]')** is the NumPy representation of a 64-bit datetime data type with nanosecond precision, which is the format used by pandas to store datetime values in the 'DateTime' index of your DataFrame.

In [104]:
data_frame['Power generated by system | (kW)'].replace(0, np.nan, inplace= True)


In [105]:
data_frame['Power generated by system | (kW)'].fillna(method= 'ffill', inplace= True)

In [106]:
data_frame.values[0]

array([1.09260e+01, 9.79103e-01, 9.01400e+00, 2.29000e+02, 3.36881e+04])

In [107]:
data_frame.values

array([[ 1.09260e+01,  9.79103e-01,  9.01400e+00,  2.29000e+02,
         3.36881e+04],
       [ 9.91900e+00,  9.79566e-01,  9.42800e+00,  2.32000e+02,
         3.72619e+04],
       [ 8.56700e+00,  9.79937e-01,  8.70000e+00,  2.36000e+02,
         3.05029e+04],
       ...,
       [ 3.01500e+00,  9.93726e-01,  1.59000e+00,  3.50000e+01,
         1.56125e+03],
       [-3.17000e-01,  9.93483e-01,  2.01200e+00,  4.50000e+01,
         1.56125e+03],
       [-1.04800e+00,  9.93414e-01,  2.13200e+00,  3.90000e+01,
         1.56125e+03]])

In [108]:
print(len(dataset))
data_frame.shape

52560


(52560, 5)

'method='ffill': This parameter specifies the fill method, which is set to 'ffill', indicating that forward fill should be used. Forward fill propagates the last valid observation forward to fill the missing values.

In [109]:
dataset = data_frame.values
train_size = int(len(dataset) * 0.7)
print(train_size)

36792


In [110]:
test_size = len(dataset) - train_size
print(test_size)

15768


In [112]:
dataset.dtype

dtype('float64')

In [120]:
dataset[0].dtype

dtype('float64')

In [116]:
dataset[0]

array([1.09260e+01, 9.79103e-01, 9.01400e+00, 2.29000e+02, 3.36881e+04])

In [117]:
dataset[0][0]

10.926

In [118]:
dataset[0][0].dtype

dtype('float64')

In [126]:
x = dataset[0 : train_size]
print(len(x))

z = print(len())

36792


In [111]:
train, test = dataset[0: ]

ValueError: ignored